In [113]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
# import warnings filter
from warnings import simplefilter
#ignore all future warnings
simplefilter(action = 'ignore', category=FutureWarning)

In [114]:
churn = pd.read_csv('Churn.csv')
churn= pd.read_csv('Churn.csv', na_values='?')
churn = churn.rename(index=str, columns={'Churn?':'Churn',"Int'l Plan":'Intl Plan'})
churn.drop(["Phone"], axis = 1, inplace=True)
#churn['Churn'] = churn['Churn'].map(lambda x: x.rstrip('.'))
churn['Churn'] = churn['Churn'].replace('False.', 0)
churn['Churn'] = churn['Churn'].replace('True.', 1)
churn = churn.dropna()
churn.head()

,Unnamed: 0,State,Account Length,Area Code,Intl Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn
3,3,OH,84,408.0,yes,no,0,299.4,71,50.90,...,88.0,5.26,196.9,89,8.86,6.6,7.0,1.78,2,0
4,4,OK,75,415.0,yes,no,0,166.7,113,28.34,...,122.0,12.61,186.9,121,8.41,10.1,3.0,2.73,3,0
5,5,AL,118,510.0,yes,no,0,223.4,98,37.98,...,101.0,18.75,203.9,118,9.18,6.3,6.0,1.70,0,0
6,6,MA,121,510.0,no,yes,24,218.2,88,37.09,...,108.0,29.62,212.6,118,9.57,7.5,7.0,2.03,3,0
9,9,WV,141,415.0,yes,yes,37,258.6,84,43.96,...,111.0,18.87,326.4,97,14.69,11.2,5.0,3.02,0,0


In [115]:
label_encoder = preprocessing.LabelEncoder()

In [116]:
churn['State'] = label_encoder.fit_transform(churn['State'])
churn['Intl Plan'] = label_encoder.fit_transform(churn['Intl Plan'])
churn['VMail Plan'] = label_encoder.fit_transform(churn['VMail Plan'])
print (churn.dtypes)

Unnamed: 0          int64
State               int32
Account Length      int64
Area Code         float64
Intl Plan           int32
VMail Plan          int32
VMail Message       int64
Day Mins          float64
Day Calls           int64
Day Charge        float64
Eve Mins          float64
Eve Calls         float64
Eve Charge        float64
Night Mins        float64
Night Calls         int64
Night Charge      float64
Intl Mins         float64
Intl Calls        float64
Intl Charge       float64
CustServ Calls      int64
Churn               int64
dtype: object


In [117]:
X = churn.drop('Churn', axis=1)  
y = churn['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0)

In [118]:
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)

In [119]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=1.0).fit(X_train, y_train)
scores = cross_val_score(clf, X, y, cv=10, scoring = 'roc_auc')
print('AUC:%0.2f' %(scores.mean()))
print('Accuracy of Logistic Regression classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of Logistic Regression classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

AUC:0.80
Accuracy of Logistic Regression classifier on training set: 0.87
Accuracy of Logistic Regression classifier on test set: 0.86


In [120]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth = 4, random_state = 0).fit(X_train, y_train)

scores = cross_val_score(clf, X, y, cv=10, scoring = 'roc_auc')
print('AUC:%0.2f' %(scores.mean()))
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

AUC:0.87
Accuracy of Decision Tree classifier on training set: 0.93
Accuracy of Decision Tree classifier on test set: 0.93


In [121]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier().fit(X_train, y_train)
scores = cross_val_score(clf, X, y, cv=10, scoring = 'roc_auc')
print('AUC:%0.2f' %(scores.mean()))
print('Accuracy of Gradient Boosting classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of Gradient Boosting classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))
# Model is overfitting, when no parameter tuning is done

AUC:0.92
Accuracy of Gradient Boosting classifier on training set: 0.98
Accuracy of Gradient Boosting classifier on test set: 0.94


In [123]:
clf = GradientBoostingClassifier(learning_rate = 0.01,n_estimators = 20,max_depth = 3,random_state = 0).fit(X_train, y_train)
scores = cross_val_score(clf, X, y, cv=10, scoring = 'roc_auc')
print('AUC:%0.2f' %(scores.mean()))
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))
# Model is perfectly fit when parameter tuning is done, on expense of AUC score which drops down to 0.84!

AUC:0.84
Accuracy of Decision Tree classifier on training set: 0.85
Accuracy of Decision Tree classifier on test set: 0.85
